In [3]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [41]:
baseURL = 'https://www.imdb.com'

## Get genre
url = "https://www.imdb.com/chart/top/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find_all('li', class_='subnav_item_main')
listGenre = [{'Name':item.text.replace('\n','').strip(),'URL':baseURL+item.a['href']} for item in results]
df_genre = pd.DataFrame(listGenre)
print ("DF genres with url:")
df_genre

DF Genres with url:


,Name,URL
0,Action,https://www.imdb.com/search/title?genres=actio...
1,Adventure,https://www.imdb.com/search/title?genres=adven...
2,Animation,https://www.imdb.com/search/title?genres=anima...
3,Biography,https://www.imdb.com/search/title?genres=biogr...
4,Comedy,https://www.imdb.com/search/title?genres=comed...
5,Crime,https://www.imdb.com/search/title?genres=crime...
6,Drama,https://www.imdb.com/search/title?genres=drama...
7,Family,https://www.imdb.com/search/title?genres=famil...
8,Fantasy,https://www.imdb.com/search/title?genres=fanta...
9,Film-Noir,https://www.imdb.com/search/title?genres=film_...


In [97]:
def GetDataPerGenre(genre, uri):
    page = requests.get(uri)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find_all('div', class_='lister-item mode-advanced')
    data = []
    for el in content:
        name = el('h3',class_='lister-item-header')[0].a.text ## get item without list
        i = el('h3',class_='lister-item-header')[0].a['href'].split('/')[2]  ## get item without list
        year = el('h3',class_='lister-item-header')[0].find('span', class_='lister-item-year').text.replace('(','').replace(')','')
        imdb_rating = el('div',class_='ratings-bar')[0].find('div', class_='ratings-imdb-rating').strong.text
        director = el('div', class_='lister-item-content')[0].find('p', class_='').find_all('a')[0].text
        #stars = el('div', class_='lister-item-content')[0].find('p', class_='').find_all('a')[1].text + ',' + 
        votes = el('div', class_='lister-item-content')[0].find('p',class_='sort-num_votes-visible').find_all('span',{"name":"nv"})[0].text
        
        ## get data of votes, Males \ Females .... <-- https://www.imdb.com/title/ID/ratings/?ref_=tt_ov_rt
        
        try:
            gross = el('div', class_='lister-item-content')[0].find('p',class_='sort-num_votes-visible').find_all('span',{"name":"nv"})[1].text
        except:
            gross = 'NaN'
            
        data.append({'Name':name,'Genre':genre,'ID':i,'Year':year,'Imdb Rating':imdb_rating,'Director':director, 'Votes':votes,'Gross':gross})
    return data

## forech listGenre .... with pages
print ('Name: '+listGenre[0]['Name'])
print ('URL: '+listGenre[0]['URL'])
temp = GetDataPerGenre(listGenre[0]['Name'], listGenre[0]['URL'])
print (len(temp))
data = pd.DataFrame(temp)
data



Name: Action
URL: https://www.imdb.com/search/title?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,
50


,Name,Genre,ID,Year,Imdb Rating,Director,Votes,Gross
0,The Dark Knight,Action,tt0468569,2008,9.0,Christopher Nolan,"2,553,055",$534.86M
1,The Lord of the Rings: The Return of the King,Action,tt0167260,2003,9.0,Peter Jackson,"1,775,505",$377.85M
2,K.G.F: Chapter 2,Action,tt10698680,2022,8.9,Prashanth Neel,"89,110",NaN
3,Everything Everywhere All at Once,Action,tt6710474,2022,8.8,Dan Kwan,"34,269",NaN
4,Inception,Action,tt1375666,2010,8.8,Christopher Nolan,"2,266,964",$292.58M
5,The Lord of the Rings: The Two Towers,Action,tt0167261,2002,8.8,Peter Jackson,"1,603,640",$342.55M
6,The Lord of the Rings: The Fellowship of the Ring,Action,tt0120737,2001,8.8,Peter Jackson,"1,796,864",$315.54M
7,The Matrix,Action,tt0133093,1999,8.7,Lana Wachowski,"1,856,877",$171.48M
8,The Empire Strikes Back,Action,tt0080684,1980,8.7,Irvin Kershner,"1,249,024",$290.48M
9,Terminator 2: Judgment Day,Action,tt0103064,1991,8.6,James Cameron,"1,067,115",$204.84M
